In [1]:
"""
프로젝트 진행 기본 코드
"""

import pandas as pd
import datetime as dt


# 날짜 차이 계산
def calculate_days_difference(date):
    NOW = dt.datetime(2022, 12, 1)
    # 현재 날짜와 입력 날짜 간의 차이를 계산
    difference = NOW - date
    return difference.days


# 데이터 파일 불러오기
def read_data(path):
    df = pd.read_csv(path)

    # 주문일자 컬럼 datetime 자료형으로 변환
    df["주문일자"] = pd.to_datetime(df["주문일자"])
    return df


# 인자로 전달한 dataframe을  csv 파일로 저장
def save_dataframe(dataframe: pd.DataFrame):
    dataframe.to_csv("RFM_report.csv")

def save_dataframe2(dataframe: pd.DataFrame):
    dataframe.to_csv("df.csv")


file_path = "./shopping.csv"

df = read_data(file_path)

In [2]:
# 데이터확인
print(df.head(10))
print(df.tail(10))

   주문번호                구매자  상품명    판매금액  결제방법                주문일자  처리상태
0     0     Fleming-Wilcox    0    5000  신용카드 2019-12-13 17:03:37  구매확정
1     1     Fleming-Wilcox    0    5000  신용카드 2019-12-16 10:04:50  구매확정
2     1     Fleming-Wilcox    1   67500  신용카드 2019-12-16 10:04:50  구매확정
3     2     Fleming-Wilcox    2   30000  신용카드 2019-12-16 17:33:43  구매확정
4     2     Fleming-Wilcox    3     750  신용카드 2019-12-16 17:33:43  구매확정
5     2     Fleming-Wilcox    4   16000  신용카드 2019-12-16 17:33:43  구매확정
6     2     Fleming-Wilcox    5    6000  신용카드 2019-12-16 17:33:43  구매확정
7     2     Fleming-Wilcox    6   20000  신용카드 2019-12-16 17:33:43  구매확정
8     3  Maldonado-Pearson    7     900  가상계좌 2019-12-17 14:29:52  주문취소
9     4     Fleming-Wilcox    8  105000  신용카드 2019-12-18 13:16:06   미결제
          주문번호                      구매자    상품명    판매금액   결제방법  \
218590  154786  Meza, Woods and Ramirez  12693    8200   신용카드   
218591  154787  Meza, Woods and Ramirez  12713    2550    포인트   
218592  15478

In [3]:
print(f"데이터 형태 : {df.shape}")
print(f"결측치 확인 : {df.isnull().sum()}")
print(f"중복데이터 확인 : {df.duplicated().value_counts()}")
print(f"데이터 자료형 : {df.info()}")
print(f"처리상태 : {df['처리상태'].unique()}")
print(f"결제방법 : {df['결제방법'].unique()}")

데이터 형태 : (218600, 7)
결측치 확인 : 주문번호     0
구매자      0
상품명      0
판매금액     0
결제방법    14
주문일자     0
처리상태     0
dtype: int64
중복데이터 확인 : False    215541
True       3059
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218600 entries, 0 to 218599
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   주문번호    218600 non-null  int64         
 1   구매자     218600 non-null  object        
 2   상품명     218600 non-null  int64         
 3   판매금액    218600 non-null  int64         
 4   결제방법    218586 non-null  object        
 5   주문일자    218600 non-null  datetime64[ns]
 6   처리상태    218600 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 11.7+ MB
데이터 자료형 : None
처리상태 : ['구매확정' '주문취소' '미결제' '후불반려' '환불완료' '환불승인' '부분환불' '부분취소' '상담취소' '교환완료'
 '상담형상품접수' '환불요청' '배송완료' '배송준비' '배송중' '시안확인요청' '결제완료']
결제방법 : ['신용카드' '가상계좌' '후불' '무통장입금' nan '신 + 포' '포인트' '신 + 적' '신 + 적 + 포' '적 + 포

In [4]:
## 결측치 처리
# 결측치가 발생한 사람
print(f"결측치가 발생한 사람 : {df[df['상품명'].isnull()].loc[:,'구매자'].unique()}")
print(
    f"결측치가 발생했을 때 처리상태 : {df[df['상품명'].isnull()].loc[:,'처리상태'].unique()}"
)
print(
    f"결측치가 발생했을 때 처리상태 : {df[df['상품명'].isnull()].loc[:,'처리상태'].unique()}"
)

결측치가 발생한 사람 : []
결측치가 발생했을 때 처리상태 : []
결측치가 발생했을 때 처리상태 : []


In [5]:
df[df["판매금액"] == 0]

,주문번호,구매자,상품명,판매금액,결제방법,주문일자,처리상태
218,88,Davis Group,72,0,신용카드,2019-12-23 17:10:24,미결제
244,101,Fleming-Wilcox,80,0,신용카드,2019-12-24 00:56:11,주문취소
246,101,Fleming-Wilcox,81,0,신용카드,2019-12-24 00:56:11,주문취소
250,101,Fleming-Wilcox,82,0,신용카드,2019-12-24 00:56:11,주문취소
252,101,Fleming-Wilcox,84,0,신용카드,2019-12-24 00:56:11,주문취소
374,131,Maldonado-Pearson,138,0,신용카드,2019-12-26 14:25:33,미결제
889,341,Porter-Vance,242,0,NaN,2020-01-07 11:14:01,미결제
904,349,Porter-Vance,242,0,신용카드,2020-01-07 12:55:01,미결제
937,360,Porter-Vance,242,0,신용카드,2020-01-07 15:16:36,미결제
1388,526,Porter-Vance,242,0,신용카드,2020-01-15 09:44:58,미결제


In [6]:
## 데이터 전처리

# 처리상태
"""
처리상태가 주문취소, 미결제, 환불완료,환불승인인 항목은 제거할 것 -> 
확실히 구매의사를 철회했다고 판단되는 항목에 한해서 제거, 후불반려는 구매의사는 있지만 개인 금전과 관련된 부분이고 부분 환불이나 취소 역시 
구매의사가 있었다고 판단.
"""

# 처리상태 전처리
df = df[
    df["처리상태"].isin(
        [
            "구매확정",
            "후불반려",
            "부분환불",
            "부분취소",
            "상담취소",
            "교환완료",
            "상담형상품접수",
            "환불요청",
            "배송완료",
            "배송준비",
            "배송중",
            "시안확인요청",
            "결제완료",
        ]
    )
].reset_index(drop=True)

# 결측치(상품명)
"""확인결과 상품명이 결측치일때 주문금액이 다르고 결제수단, 구매확정 등 다른 요소가 여전히 존재하기에 본 분석에서는 제거할 수 없는 것으로 확인했고 상품명이 결측치인 경우 unidentify라는 이름으로 대체하여 분석 진행"""

# 상품명 전처리
df["상품명"] = df["상품명"].fillna("unidentify")

# 결제방법
"""
위 과정을 다 거치고 남은 결제 방법이 결측치인것을 확인해본 결과 구매자가 구매를 한 항목이라기 보다는 사은품이나 가맹용 제품인 경우, 즉 이 값들은 제거한다
"""

# 결제방법 전처리
df = df.dropna()


print(f"최종 결측치 확인 : {df.isnull().sum()}")


# 추가 전처리
"""판매금액이 0원인 것은 부분취소 혹은 환불이거나 학습 교구등인 경우가 있는데 실제 금액을 다루기가 어려움으로 제거하고 분석"""
df = df[df["판매금액"] != 0]

# 최종데이터
print(f"분석에 사용할 최종 데이터 형태 : {df.shape}")

최종 결측치 확인 : 주문번호    0
구매자     0
상품명     0
판매금액    0
결제방법    0
주문일자    0
처리상태    0
dtype: int64
분석에 사용할 최종 데이터 형태 : (161796, 7)


In [7]:
## RFM_result 생성

"""
- 구매자 (그룹화 기준)
- recency (최근 구매일)
- frequency (총 구매 횟수)
- monetary (총 구매 금액)
- R_score (Recency 점수 1 ~ 5)
- F_score (Frequency 점수 1 ~ 5)
- M_score (Monetary 점수 1 ~ 5)
- grade (RFM 점수를 기준으로 분류한 5개 그룹 중 하나)
"""

# 구매자 그룹
grouped = df.groupby("구매자")

recency = grouped["주문일자"].max()
frequency = grouped["주문번호"].count()
monetary = grouped["판매금액"].sum()

# R_score
daylen = []
for i in range(len(recency)):
    a = calculate_days_difference(recency[i])
    daylen.append(a)

R_score = []
for v in daylen:
    if v <= 30:
        v = 5
    elif v <= 90:
        v = 4
    elif v <= 180:
        v = 3
    elif v <= 365:
        v = 2
    else:
        v = 1
    R_score.append(v)

print(R_score)

# F_score
F_score = pd.qcut(
    frequency,
    q=[0, 0.3, 0.5, 0.7, 0.9, 1.0],
    labels=["1", "2", "3", "4", "5"],
)

print(F_score)


# M_score
M_score = pd.qcut(
    monetary,
    q=[0, 0.3, 0.5, 0.7, 0.9, 1.0],
    labels=["1", "2", "3", "4", "5"],
)

print(M_score)

[5, 5, 5, 5, 2, 4, 4, 3, 5, 5, 1, 1, 5, 5, 2, 4, 2, 5, 5, 1, 4, 5, 4, 5, 2, 5, 4, 2, 2, 1, 1, 5, 5, 1, 1, 5, 5, 1, 5, 5, 3, 1, 5, 4, 5, 5, 2, 2, 5, 3, 5, 2, 4, 1, 4, 5, 5, 4, 5, 3, 2, 4, 1, 5, 4, 5, 5, 1, 3, 1, 2, 5, 5, 1, 3, 2, 2, 5, 5, 1, 3, 5, 3, 1, 2, 1, 4]
구매자
Arnold Ltd                       4
Barnes, Martinez and Romero      4
Bell, Ortega and Ray             5
Black-Williams                   4
Brooks, Horton and Ruiz          3
                                ..
Williams, Garcia and Moody       1
Williams-Williams                1
Willis-Hurley                    2
Wilson, Conrad and Richardson    2
Wright LLC                       4
Name: 주문번호, Length: 87, dtype: category
Categories (5, object): ['1' < '2' < '3' < '4' < '5']
구매자
Arnold Ltd                       3
Barnes, Martinez and Romero      2
Bell, Ortega and Ray             4
Black-Williams                   4
Brooks, Horton and Ruiz          3
                                ..
Williams, Garcia and Moody       2
Willia

C:\Users\user\AppData\Local\Temp\ipykernel_17164\556522099.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a = calculate_days_difference(recency[i])


In [8]:
## 신규 데이터 프레임 형성
RFM_result = pd.DataFrame(data=recency)
RFM_result["frequency"] = frequency.values
RFM_result["monetary"] = monetary.values
RFM_result["R_score"] = R_score
RFM_result["F_score"] = F_score.values
RFM_result["M_score"] = M_score.values
RFM_result["Grade"] = ""

RFM_result

,주문일자,frequency,monetary,R_score,F_score,M_score,Grade
구매자,,,,,,,
Arnold Ltd,2022-11-02 06:55:28,244,5244100,5,4,3,
"Barnes, Martinez and Romero",2022-11-07 19:13:54,220,2419680,5,4,2,
"Bell, Ortega and Ray",2022-11-03 21:21:28,2432,16124800,5,5,4,
Black-Williams,2022-11-07 18:19:11,2277,24925990,5,4,4,
"Brooks, Horton and Ruiz",2022-02-16 14:03:07,133,4326600,2,3,3,
...,...,...,...,...,...,...,...
"Williams, Garcia and Moody",2022-07-12 14:24:45,2,1152000,3,1,2,
Williams-Williams,2020-05-13 16:17:38,12,286780,1,1,1,
Willis-Hurley,2021-12-13 00:24:35,24,447800,2,2,1,


## **Grade** 나누기
### Vip 
    5,5,5
    5,4,5
    4,5,5
    4,4,5
    R이 4 이상이면서 F,M 합이 9이상(단 M이 5일것)
### 이탈우려고객
    R이 2,3이면서 F,M의 합이 5이상
### 이탈고객
    R이 1이면서 F,M의 합이 5이상
### 신경써야할고객(이탈위험, 구매빈도에 비해 지출액이 적은 고객)
    R이 4이상이면서 F,M의 합이 6이상
### 신규고객
    R과 F가 1인 고객

In [9]:
for i in range(len(RFM_result)):
    if RFM_result.iloc[i, 3] >= 4:
        if int(RFM_result.iloc[i, 4]) >= 4:
            if int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 9:
                RFM_result.iloc[i, 6] = "Vip"
            elif int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 6:
                RFM_result.iloc[i, 6] = "신경써야할고객"
        else:
            RFM_result.iloc[i, 6] = "신경써야할고객"
    elif RFM_result.iloc[i, 3] >= 2:
        if int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 5:
            RFM_result.iloc[i, 6] = "이탈우려고객"
        else:
            RFM_result.iloc[i, 6] = "신경써야할고객"
    elif RFM_result.iloc[i, 3] == 1:
        if int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 5:
            RFM_result.iloc[i, 6] = "이탈고객"
        elif int(RFM_result.iloc[i, 5]) == 1:
            RFM_result.iloc[i, 6] = "신규고객"
        else:
            RFM_result.iloc[i, 6] = "신경써야할고객"


# 이탈우려고객


# int(RFM_result.iloc[i,4])+int(RFM_result.iloc[i,5])

In [10]:
save_dataframe(RFM_result)

In [11]:
save_dataframe2(df)